# 02 — Overview (NLP) · Film Greenlight Recommender

**Objetivo** — Usar a coluna *Overview* como fonte textual para inferir **gênero** via
TF-IDF e classificadores lineares (baseline multilabel), e salvar artefatos para uso no pipeline.

**Entradas** — `data/raw/imdb.csv`  
**Saídas** —
- Modelos: `models/overview_tfidf_svc.pkl`, `models/overview_tfidf_logreg.pkl`, `models/tfidf.pkl`
- Features: `data/processed/features_text_csr.npz`, `data/processed/tfidf_vocab.parquet`
- Relatórios: `reports/nlp_per_class_svc.csv`, `reports/nlp_per_class_logreg.csv`

**Escopo**
- Preparação: remoção de nulos em `Overview` e `Genre`; construção de `Genre_list` (multilabel).
- Filtro de classes: manter gêneros com **≥ 50** ocorrências para reduzir classes raras.
- Representação: TF-IDF de *Overview* (`ngram_range=(1,3)`, `max_features=20000`, `min_df=2`, `max_df=0.95`,
`strip_accents='ascii'`, `stop_words='english'`).
- Baselines: One-vs-Rest com **LinearSVC** e **Logistic Regression** (solver `saga`, `class_weight='balanced'`).
- Avaliação: **F1-micro**, **F1-macro** e relatório **por classe**; exportação das tabelas em CSV.
- Exportação de artefatos: vetorizador, matriz esparsa e vocabulário para integração posterior.
- (Opcional) *Threshold tuning* por classe usando probabilidades da Logistic Regression.

Autora: *Ana Luiza Gomes Vieira* · Execução: *Set/2025*

## Setup

In [ ]:
# Bibliotecas padrão
from collections import Counter
from pathlib import Path
import warnings

# Bibliotecas de terceiros
import joblib
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.svm import LinearSVC

In [21]:
# Configurações globais
SEED = 42
PROJECT_ROOT = Path.cwd().parent
DATA_RAW = PROJECT_ROOT / "data" / "raw" / "imdb.csv"
MODELS_DIR = PROJECT_ROOT / "models"
REPORTS_DIR = PROJECT_ROOT / "reports"
FEATURES_DIR = PROJECT_ROOT / "data" / "processed"
for p in [MODELS_DIR, REPORTS_DIR, FEATURES_DIR]:
    p.mkdir(parents=True, exist_ok=True)

assert DATA_RAW.exists(), f"Arquivo não encontrado: {DATA_RAW}"

warnings.filterwarnings("ignore")

In [3]:
df = pd.read_csv(DATA_RAW)
df = df.dropna(subset=["Overview", "Genre"]).copy()
df["Genre_list"] = df["Genre"].str.split(", ").apply(lambda xs: [x.strip() for x in xs])

In [4]:
df.head()

,Unnamed: 0,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross,Genre_list
0,1,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411","[Crime, Drama]"
1,2,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444","[Action, Crime, Drama]"
2,3,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000","[Crime, Drama]"
3,4,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000","[Crime, Drama]"
4,5,The Lord of the Rings: The Return of the King,2003,U,201 min,"Action, Adventure, Drama",8.9,Gandalf and Aragorn lead the World of Men agai...,94.0,Peter Jackson,Elijah Wood,Viggo Mortensen,Ian McKellen,Orlando Bloom,1642758,"377,845,905","[Action, Adventure, Drama]"


## 1. Filtro de Gêneros (Frequência Mínima)

In [ ]:
MIN_COUNT = 50
counts = pd.Series([g for xs in df["Genre_list"] for g in xs]).value_counts()
keep = set(counts[counts >= MIN_COUNT].index)
df["Genre_list"] = df["Genre_list"].apply(lambda xs: [g for g in xs if g in keep])
df = df[df["Genre_list"].map(len) > 0].reset_index(drop=True)

## 2. Alvo Multilabel e Divisão Treino/Teste

In [ ]:
X = df["Overview"].values
mlb = MultiLabelBinarizer()
Y = mlb.fit_transform(df["Genre_list"])
class_names = mlb.classes_

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, random_state=SEED
)

## 3. Representação TF-IDF

In [ ]:
svc_pipe = Pipeline([
    ("tfidf", TfidfVectorizer(
        stop_words="english",
        strip_accents="ascii",
        max_features=20000,
        ngram_range=(1,3),
        min_df=2,
        max_df=0.95,
    )),
    ("clf", OneVsRestClassifier(LinearSVC(random_state=SEED)))
])

In [9]:
svc_pipe.fit(X_train, Y_train)
Y_pred_svc = svc_pipe.predict(X_test)

## 4. Baselines de classificação

### 4.1 LinearSVC (One-vs-Rest)

In [10]:
print("[LinearSVC] F1-micro:", f1_score(Y_test, Y_pred_svc, average="micro"))
print("[LinearSVC] F1-macro:", f1_score(Y_test, Y_pred_svc, average="macro"))
print("\n[LinearSVC] Relatório por gênero:\n")
print(classification_report(Y_test, Y_pred_svc, target_names=class_names, zero_division=0))

[LinearSVC] F1-micro: 0.48043184885290147
[LinearSVC] F1-macro: 0.2267174691676371

[LinearSVC] Relatório por gênero:

              precision    recall  f1-score   support

      Action       0.57      0.24      0.34        33
   Adventure       0.60      0.43      0.50        28
   Animation       0.50      0.12      0.19        17
   Biography       1.00      0.09      0.16        23
      Comedy       0.38      0.15      0.21        41
       Crime       0.61      0.33      0.43        42
       Drama       0.79      0.86      0.82       148
      Family       1.00      0.10      0.18        10
     Fantasy       0.00      0.00      0.00        18
     History       1.00      0.08      0.15        12
     Mystery       0.50      0.08      0.13        26
     Romance       0.14      0.03      0.05        35
      Sci-Fi       1.00      0.09      0.17        11
    Thriller       0.14      0.04      0.06        26
         War       0.00      0.00      0.00         9

   micro avg   

### 4.2 Logistic Regression (One-vs-Rest)

In [ ]:
logreg_pipe = Pipeline([
    ("tfidf", TfidfVectorizer(
        stop_words="english",
        strip_accents="ascii",
        max_features=20000,
        ngram_range=(1,3),
        min_df=2,
        max_df=0.95,
    )),
    ("clf", OneVsRestClassifier(
        LogisticRegression(
            solver="saga", max_iter=2000, class_weight="balanced", n_jobs=-1, random_state=SEED
        )
    )),
])

In [22]:
logreg_pipe.fit(X_train, Y_train)
Y_pred_lr = logreg_pipe.predict(X_test)

In [13]:
print("\n[LogReg] F1-micro:", f1_score(Y_test, Y_pred_lr, average="micro"))
print("[LogReg] F1-macro:", f1_score(Y_test, Y_pred_lr, average="macro"))
print("\n[LogReg] Relatório por gênero:\n")
print(classification_report(Y_test, Y_pred_lr, target_names=class_names, zero_division=0))


[LogReg] F1-micro: 0.42560865644724977
[LogReg] F1-macro: 0.3029859953780284

[LogReg] Relatório por gênero:

              precision    recall  f1-score   support

      Action       0.52      0.42      0.47        33
   Adventure       0.45      0.61      0.52        28
   Animation       0.22      0.47      0.30        17
   Biography       0.56      0.22      0.31        23
      Comedy       0.35      0.34      0.35        41
       Crime       0.53      0.50      0.51        42
       Drama       0.80      0.77      0.79       148
      Family       0.33      0.10      0.15        10
     Fantasy       0.09      1.00      0.17        18
     History       0.11      0.33      0.17        12
     Mystery       0.38      0.19      0.26        26
     Romance       0.45      0.26      0.33        35
      Sci-Fi       0.00      0.00      0.00        11
    Thriller       0.25      0.23      0.24        26
         War       0.00      0.00      0.00         9

   micro avg       0.37

## 5. Métrica por classe e relatórios

In [ ]:
def per_class_f1(y_true, y_pred, names):
    from sklearn.metrics import precision_recall_fscore_support
    p, r, f1, s = precision_recall_fscore_support(y_true, y_pred, average=None, zero_division=0)
    out = pd.DataFrame({"class": names, "precision": p, "recall": r, "f1": f1, "support": s})
    return out.sort_values("f1", ascending=False)

svc_per_class = per_class_f1(Y_test, Y_pred_svc, class_names)
lr_per_class = per_class_f1(Y_test, Y_pred_lr, class_names)

svc_per_class.to_csv(REPORTS_DIR / "nlp_per_class_svc.csv", index=False)
lr_per_class.to_csv(REPORTS_DIR / "nlp_per_class_logreg.csv", index=False)


## 6. Exportação de artefatos

In [15]:
joblib.dump(svc_pipe, MODELS_DIR / "overview_tfidf_svc.pkl")
joblib.dump(logreg_pipe, MODELS_DIR / "overview_tfidf_logreg.pkl")

['c:\\Users\\analu\\Documents\\film-greenlight-recommender\\models\\overview_tfidf_logreg.pkl']

In [ ]:
tfidf = TfidfVectorizer(
    stop_words="english", strip_accents="ascii",
    max_features=20000, ngram_range=(1,3),
    min_df=2, max_df=0.95
).fit(df["Overview"].values)

In [17]:
import scipy.sparse as sp
X_tfidf_full = tfidf.transform(df["Overview"].values)

In [18]:
from scipy import sparse
sparse.save_npz(FEATURES_DIR / "features_text_csr.npz", X_tfidf_full)
pd.DataFrame({"vocab": tfidf.get_feature_names_out()}).to_parquet(FEATURES_DIR / "tfidf_vocab.parquet", index=False)
joblib.dump(tfidf, MODELS_DIR / "tfidf.pkl")

print("\nArtefatos salvos em:", MODELS_DIR, "e", FEATURES_DIR)


Artefatos salvos em: c:\Users\analu\Documents\film-greenlight-recommender\models e c:\Users\analu\Documents\film-greenlight-recommender\data\processed


## 7. Threshold tuning

In [19]:
proba = logreg_pipe.predict_proba(X_test)
thr = np.full(proba.shape[1], 0.5)
Y_pred_thr = (proba >= thr).astype(int)
print("F1-macro @0.50:", f1_score(Y_test, Y_pred_thr, average="macro"))

F1-macro @0.50: 0.3029859953780284


#### Análise de *Overview* (NLP)
A coluna `Overview` foi processada com **TF-IDF (1–3-grams)** e testada em dois baselines de classificação multilabel.

- **Modelos avaliados**  
  - **LinearSVC (One-vs-Rest)**  
    - F1-micro ≈ **0.48**  
    - F1-macro ≈ **0.23**  
    - Muito bom em `Drama` (F1=0.82), razoável em `Adventure` (0.50) e `Crime` (0.43).  
    - Fraco ou nulo em gêneros menores como `Fantasy`, `War`, `Thriller`.

  - **Logistic Regression (One-vs-Rest, solver saga)**  
    - F1-micro ≈ **0.43**  
    - F1-macro ≈ **0.30**  
    - Melhor recall em algumas classes minoritárias (`Animation`, `Biography`), o que elevou o macro-F1.  
    - `Drama` manteve desempenho robusto (F1=0.79).  
    - Classes muito pequenas continuaram com desempenho baixo (`Sci-Fi`, `War`).

- **Insights do NLP**  
  - As sinopses trazem sinais semânticos claros: termos como *war, battle, soldier* indicam gêneros bélicos; *love, family, heart* aparecem em dramas/romances; *alien, space, world* em Sci-Fi.  
  - O **desbalanceamento severo** dos gêneros (ex.: `Drama` >> `War`) limita o macro-F1, pois o modelo aprende bem apenas as classes mais frequentes.  
  - O **Logistic Regression** mostrou-se mais equilibrado em termos de macro-F1, além de permitir uso de probabilidades para futuros ajustes de limiar por classe.

---

#### Recomendações baseadas no EDA + NLP
- **Para bilheteria**: apostar em gêneros comerciais (Action, Adventure, Sci-Fi, Animation), certificados PG/PG-13 e elencos/diretores com histórico comprovado.  
- **Para crítica**: gêneros familiares e históricos tendem a garantir notas mais altas da crítica especializada.  
- **Para recomendações gerais**: usar métricas ponderadas como o **IMDb Weighted Rating (WR)**, que equilibra nota e número de votos.  
- **Para modelagem de gênero via Overview**: baseline já mostra que o texto é informativo; próximos passos poderiam incluir *threshold tuning*, *SVD* para reduzir dimensionalidade e uso de embeddings pré-treinados (BERT).

---

#### Conclusão
- O dataset é consistente e oferece insumos para entender **popularidade, receita e qualidade crítica**.  
- O texto do `Overview` contribui para prever gêneros, mesmo com limitações.  
- As diferenças entre F1-micro e F1-macro ressaltam a necessidade de técnicas para lidar com **classes desbalanceadas**.  
- Combinando **variáveis tabulares + Overview**, o sistema de recomendação pode ser mais robusto, equilibrando retorno comercial e aprovação crítica.